<a href="https://colab.research.google.com/github/efecantepe/IE423/blob/main/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount("/content/drive")

dfJokes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/JokeText.csv')
dfUsr1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UserRatings1.csv')

dfJokes.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [4]:
dfUsr1.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


# 1. Content Based Filtering

In [7]:
dfJokes.shape

(100, 2)

## Build Model

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(analyzer="word", ngram_range=(1,2), stop_words="english")
tfidf_matrix = model.fit_transform(dfJokes['JokeText'])
tfidf_matrix.shape

(100, 3774)

### Calculating Cosine Similarity And Prediction

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

def get_recommendations(JokeId):
    sim_scores = list(enumerate(cosine_sim[JokeId]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    joke_indices = [i[0] for i in sim_scores]
    return dfJokes.iloc[joke_indices]

get_recommendations(0).head(10)



,JokeId,JokeText
86,86,"A man, recently completing a routine physical ..."
67,67,A man piloting a hot air balloon discovers he ...
87,87,A Czechoslovakian man felt his eyesight was gr...
75,75,There once was a man and a woman that both go...
31,31,A man arrives at the gates of heaven. St. Pete...
38,38,What is the difference between men and women:\...
55,55,A man and Cindy Crawford get stranded on a des...
80,80,An Asian man goes into a New York CityBank to ...
32,32,What do you call an American in the finals of ...
3,3,Q. What's the difference between a man and a t...


With this function when we entered the joke id we will get the similar jokes based on their content.

## Collaborative Filtering

In [16]:

## Converting in more readble form
dfUsr1 = pd.melt(dfUsr1,id_vars=['JokeId'],var_name='User',value_name='Rating')
dfUsr1


,JokeId,User,Rating
0,0,User1,5.10
1,1,User1,4.90
2,2,User1,1.75
3,3,User1,-4.17
4,4,User1,5.15
...,...,...,...
3670995,95,User36710,NaN
3670996,96,User36710,NaN
3670997,97,User36710,NaN
3670998,98,User36710,NaN


In [18]:
dfUsr1 = dfUsr1.dropna(subset = ['Rating'])
dfUsr1

## Since rating is what we are looking for we dropped null ratings.

,JokeId,User,Rating
0,0,User1,5.10
1,1,User1,4.90
2,2,User1,1.75
3,3,User1,-4.17
4,4,User1,5.15
...,...,...,...
3670967,67,User36710,3.59
3670968,68,User36710,5.39
3670969,69,User36710,4.71
3670980,80,User36710,0.97


In [20]:
!pip3 install scikit-surprise #or !conda install -c conda-forge scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0,5))
X = Dataset.load_from_df(dfUsr1, reader)
X_train, X_test = train_test_split(X, test_size=.25)

In [21]:
from surprise import SVD

mdlSVD = SVD()

mdlSVD.fit(X_train)
test_pred = mdlSVD.test(X_test)


from surprise import accuracy

accuracy.rmse(test_pred)

RMSE: 4.6215


4.621507553636597

## Prediction

In [24]:
dfUsr1[dfUsr1['User'] == 'User1']


,JokeId,User,Rating
0,0,User1,5.10
1,1,User1,4.90
2,2,User1,1.75
3,3,User1,-4.17
4,4,User1,5.15
...,...,...,...
95,95,User1,6.31
96,96,User1,-4.95
97,97,User1,-0.19
98,98,User1,3.25


In [27]:
mdlSVD.predict(1, 82)


Prediction(uid=1, iid=82, r_ui=None, est=0.2107465360874533, details={'was_impossible': False})